In [ ]:
import nltk
nltk.download('gutenberg')

nltk.corpus.gutenberg.fileids()

In [ ]:
bible = nltk.corpus.gutenberg.words("bible-kjv.txt")



In [4]:
import pandas as pd
nltk.download('wordnet')

df = pd.read_csv('SimLex-999.txt', delimiter = "\t")
df

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


,word1,word2,POS,SimLex999,conc(w1),conc(w2),concQ,Assoc(USF),SimAssoc333,SD(SimLex)
0,old,new,A,1.58,2.72,2.81,2,7.25,1,0.41
1,smart,intelligent,A,9.20,1.75,2.46,1,7.11,1,0.67
2,hard,difficult,A,8.77,3.76,2.21,2,5.94,1,1.19
3,happy,cheerful,A,9.55,2.56,2.34,1,5.85,1,2.18
4,hard,easy,A,0.95,3.76,2.07,2,5.82,1,0.93
...,...,...,...,...,...,...,...,...,...,...
994,join,acquire,V,2.85,2.86,2.93,2,0.00,0,0.99
995,send,attend,V,1.67,2.70,3.17,2,0.00,0,1.44
996,gather,attend,V,4.80,2.75,3.17,2,0.00,0,1.97
997,absorb,withdraw,V,2.97,3.11,3.04,2,0.00,0,1.75


In [18]:
# TODO: fix lib errors 
from nltk.corpus import wordnet as wn
from nltk.corpus import genesis
import numpy as np
from scipy import stats

genesis_corp = wn.ic(wn,False,0.0)
sims = []
processed_scores =[]
for word1, word2, pos_tag, simlex_score in zip(df.word1, df.word2, df.POS, df.SimLex999):
  s1 = wn.synset("{}.{}.01".format(word1, pos_tag.lower()))
  s2 = wn.synset("{}.{}.01".format(word2, pos_tag.lower()))
  try:
    p_sim=s1.path_similarity(s2)
    lch_sim=s1.lch_similarity(s2)
    wup_sim=s1.wup_similarity(s2)
    res_sim=s1.res_similarity(s2,genesis_corp)
    jcn_sim=s1.jcn_similarity(s2,genesis_corp)
    lin_sim=s1.lin_similarity(s2,genesis_corp)
    res =[p_sim,lch_sim,wup_sim,res_sim,jcn_sim,lin_sim]
    for metric in res:
      if metric is None:
        raise ValueError('none encountered')
 
    sims.append(res)
    processed_scores.append(simlex_score)
  except:
    #just skip exceptions
    continue

sims = np.asarray(sims)
processed_scores= np.asarray(processed_scores)
results =[]
n = sims.shape[1]
for i in range(n):
  results.append(stats.pearsonr(sims[:,i],processed_scores))

metrics =['path_sim','lch_sim','wup_sim','res_sim','jcn_sim','lin_sim']
for m, acc in zip(metrics, results):
  print("metric:{}, pearson corr:{}".format(m,acc))

metric:path_sim, pearson corr:(0.43888786796125767, 4.15584005221031e-43)
metric:lch_sim, pearson corr:(0.4339421864617497, 4.461004522769442e-42)
metric:wup_sim, pearson corr:(0.3505600823618937, 4.491014495455419e-27)
metric:res_sim, pearson corr:(0.35618214487021166, 5.915965125482585e-28)
metric:jcn_sim, pearson corr:(0.2174579061586609, 5.772875634331321e-11)
metric:lin_sim, pearson corr:(0.387265187401865, 3.764942376959388e-33)
